In [18]:
import pandas as pd
import numpy as np
import pickle

In [19]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [20]:
simu_sess_num = 1000
n = 12
wordpool = np.arange(1,1639)
pos_rep = [1,3,5]

In [21]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):
    tmp_df_study = pd.DataFrame()
    tmp_df_test1 = pd.DataFrame()
    tmp_df_test2 = pd.DataFrame()
    
    # choose present words
    pres_words = rng.choice(wordpool, 2*n, replace = False)
    pres_words = np.reshape(pres_words, (n,2)).tolist()
    pair_idx = np.arange(1,13) + sess * n
    pres_rep = rng.permutation(pos_rep * 4)

    pres_items = []
    for i in range(n):
        word = pres_words[i]
        rep = pres_rep[i]
        for _ in range(rep):
            pres_items.append(word)
    pres_items = np.array(pres_items)

    pres_items = rng.permutation(pres_items)
    check_rep_items = np.sum(np.abs(pres_items[1::] - pres_items[0:-1]),axis=1) # check no twice pres
    while np.isin(0,check_rep_items).item():
        pres_items = rng.permutation(pres_items)
        check_rep_items = np.sum(np.abs(pres_items[1::] - pres_items[0:-1]),axis=1)
    
    tmp_df_study['study_itemno1'] = pres_items[:,0]
    tmp_df_study['study_itemno2'] = pres_items[:,1]
    tmp_df_study['study_item1'] = items[pres_items[:,0]-1]
    tmp_df_study['study_item2'] = items[pres_items[:,1]-1]
    tmp_df_study['session'] = sess
    df_study = df_study.append(tmp_df_study)
    
    # choose test words
    test_order = rng.permutation([[1,1]] * int(n/4) + [[1,2]] * int(n/4) + [[2,1]] * int(n/4) + [[2,2]] * int(n/4))
    test1_order = test_order[:,0]
    test2_order = test_order[:,1]
    pres_words = np.array(pres_words)
    test1_probe = np.array([pres_words[i,test1_order[i]-1] for i in range(n)])
    test1_ans = np.array([pres_words[i,2-test1_order[i]] for i in range(n)])
    test2_probe = np.array([pres_words[i,test2_order[i]-1] for i in range(n)])
    test2_ans = np.array([pres_words[i,2-test2_order[i]] for i in range(n)])
    
    # test1
    probe_ans_rep = list(zip(test1_probe, test1_ans, test1_order, pres_rep, pair_idx))
    probe_ans_rep = rng.permutation(probe_ans_rep)
    test1_probe, test1_ans, test1_order, test1_rep, test1_idx = zip(*probe_ans_rep)
    test1_probe, test1_ans, test1_order, test1_rep, test1_idx = np.array(test1_probe), np.array(test1_ans), np.array(test1_order), np.array(test1_rep), np.array(test1_idx)
    
    tmp_df_test1['test_itemno'] = test1_probe
    tmp_df_test1['test_item'] = items[test1_probe-1]
    tmp_df_test1['correct_ans'] = test1_ans
    tmp_df_test1['order'] = test1_order
    tmp_df_test1['rep'] = test1_rep
    tmp_df_test1['test'] = 1
    tmp_df_test1['session'] = sess
    tmp_df_test1['test_pos'] = np.arange(1,13)
    tmp_df_test1['pair_idx'] = test1_idx
    df_test = df_test.append(tmp_df_test1)

    probe_ans_rep = list(zip(test2_probe, test2_ans, test2_order, pres_rep, pair_idx))
    probe_ans_rep = rng.permutation(probe_ans_rep)
    test2_probe, test2_ans, test2_order, test2_rep, test2_idx = zip(*probe_ans_rep)
    test2_probe, test2_ans, test2_order, test2_rep, test2_idx = np.array(test2_probe), np.array(test2_ans), np.array(test2_order), np.array(test2_rep), np.array(test2_idx)

    tmp_df_test2['test_itemno'] = test2_probe
    tmp_df_test2['test_item'] = items[test2_probe-1]
    tmp_df_test2['correct_ans'] = test2_ans
    tmp_df_test2['order'] = test2_order
    tmp_df_test2['rep'] = test2_rep
    tmp_df_test2['test'] = 2
    tmp_df_test2['session'] = sess
    tmp_df_test2['test_pos'] = np.arange(13,25)
    tmp_df_test2['pair_idx'] = test2_idx
    df_test = df_test.append(tmp_df_test2)

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [22]:
df_study.head(36)

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,153,736,BOTTLE,INDIAN,0
1,145,1198,BODY,ROBE,0
2,1391,1059,STRAWBERRY,PICKLE,0
3,140,838,BLUEBERRY,LOCK,0
4,1132,702,PROTON,HELMET,0
5,819,1251,LIGHTER,SCOTCH,0
6,1132,702,PROTON,HELMET,0
7,819,1251,LIGHTER,SCOTCH,0
8,607,856,FRIES,MAID,0
9,1132,702,PROTON,HELMET,0


In [23]:
df_test.head(48)

,test_itemno,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,327,CLOTHES,210,2,1,1,0,1,11
1,736,INDIAN,153,2,5,1,0,2,6
2,140,BLUEBERRY,838,1,1,1,0,3,1
3,1391,STRAWBERRY,1059,1,1,1,0,4,5
4,819,LIGHTER,1251,1,5,1,0,5,8
5,299,CHILD,1282,1,5,1,0,6,4
6,856,MAID,607,2,3,1,0,7,10
7,1132,PROTON,702,1,5,1,0,8,3
8,1372,STAPLER,1169,2,3,1,0,9,12
9,1198,ROBE,145,2,3,1,0,10,7


In [27]:
# save
with open('../Data/simu6b_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)